In [1]:
#Class weitht
'''Pytorch:
多分类：torch.nn.CrossEntropyLoss(weight=...); 二分类/多标签：torch.nn.BCEWithLogitsLoss(pos_weight=...)
Tensorflow2/Keras:
多分类：tf.nn.weighted_cross_entropy_with_logits(pos_weight=...); 二分类或多分类：model.fit(class_weight=...)'''

#Sample weight
'''Pytorch:
多标签: torch.nn.BCEWithLogitsLoss(weight=...)
Tensorflow2/Keras:
model.fit(sample_weight=...)
'''
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold,StratifiedKFold

from sklearn import metrics
from imblearn.metrics import geometric_mean_score

from sklearn.ensemble import RandomForestClassifier

from sklearn.utils import class_weight

ds = pd.read_csv("ahc.csv")
ds.head()

,1,2,3,4,5,6,7,8,9,10,...,42,43,44,45,46,47,48,49,50,symptom
0,0.385714,-0.730334,-0.139520,0.885217,0.185512,-0.092579,1.675729,-0.686223,0.531079,1.260273,...,0.310443,0.685676,0.743884,-1.707476,0.409856,0.234790,0.124626,-0.304451,-0.098937,1
1,-1.139784,0.202991,-0.172729,2.288758,0.832926,-1.270105,-0.541457,-0.299268,-0.662836,-1.111758,...,0.347890,0.022021,-0.588785,-2.409142,-0.092867,0.687847,-1.272507,0.852925,-0.070433,1
2,0.301588,-1.216219,-0.320328,1.682189,1.250116,-1.544504,-0.439693,0.573244,-0.466864,0.410886,...,0.699880,0.722700,-0.565397,-1.177416,1.832512,-1.763520,1.174136,-0.110950,-0.721374,1
3,0.610831,-0.224714,3.514447,0.002929,-2.178927,1.008115,-0.221103,0.152922,0.782051,-0.069084,...,0.100465,1.928904,-0.030931,0.060462,-1.101277,0.496262,-0.477511,1.070145,0.226384,1
4,0.683467,0.778225,-0.244681,-0.104264,-0.153930,1.351049,0.544877,1.948468,-0.672722,-0.002173,...,0.068522,0.431119,1.112156,0.008301,-0.151477,1.208057,-1.728618,-0.297925,-2.181558,1


In [2]:
# divide into the training and test data sets.
tr = ds.columns[:-1]


#X_train,X_test,y_train,y_test = model_selection.train_test_split(ds[tr], ds.churn, test_size=0.25,random_state=67)


kf = KFold(n_splits=10,shuffle=False)

In [11]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
total = 2868
pos = 692
neg = 2176
print(f'total: {total}', f'neg: {neg}', f'pos: {pos}')
weight_for_0 = (1 / neg) *(total)/2.0
weight_for_1 = (1 / pos) *(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

#or using sklearn.utils
# from sklearn.utils import class_weight

# class_weights = class_weight.compute_class_weight('balanced',
#                                                  np.unique(y_train),
#                                                  y_train)

# model.fit(X_train, y_train, class_weight=class_weights)

total: 2868 neg: 2176 pos: 692
Weight for class 0: 0.66
Weight for class 1: 2.07


In [8]:
#class weight
def Catclass(clas, X, target):
    acc = []
    rec = []
    f1 = []
    gm = []
    mae = []
    for train, test in kf.split(X,target): 
        X_train = X.loc[train]
        y_train = target.loc[train]    
        
    
        
        #UNDERSAMPLING
        
        #TomekLinks
        #utl = TomekLinks() 
        #usa_X,usa_y = utl.fit_sample(X_train, y_train)
        
        
      
        # The proportion of categories before resampling
        #print(y_train.value_counts()/len(y_train))
        # The proportion of categories after resampling
        #print('After resampling:')
        #print(pd.Series(usa_y).value_counts()/len(usa_y))
       
        
        
        X_test = X.loc[test]
        y_test = target.loc[test]

        #accuracy of model prediction and other metrics
       
        #clas.fit(usa_X,usa_y) # undersampling 
        
        clas.fit(X_train, y_train)
        pred = clas.predict(X_test)
        print(metrics.classification_report(y_test, pred))
        acc.append(metrics.accuracy_score(y_test, pred))
        rec.append(metrics.recall_score(y_test, pred))
        f1.append(metrics.f1_score(y_test, pred)) 
        gm.append(geometric_mean_score(y_test, pred))
        #mae.append(mean_absolute_error(y_test, pred))
    print('Modelprediction-Accuracy: %.4f; Standard Deviation: %.4f' % (np.mean(acc), np.std(acc)))
    print('Recall: %.4f; Standard Deviation: %.4f' % (np.mean(rec), np.std(rec)))
    print('F1-score: %.4f; Standard Deviation: %.4f' % (np.mean(f1), np.std(f1)))
    print('G-means: %.4f; Standard Deviation: %.4f' % (np.mean(gm), np.std(gm))) 

    #print('MAE: %.4f; Standard Deviation: %.4f' % (np.mean(mae), np.std(mae)))
#     results = cross_val_score(clas, X, target, cv=kf, scoring='neg_mean_absolute_error')
#     print(results)
#     print("Accuracy: %.4f (%.4f)" % (results.mean(), results.std()))
    

    

In [10]:
Catclass(RandomForestClassifier(random_state = 67, class_weight = 'balanced'), ds[tr],ds.symptom)

C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defi

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00     287.0

    accuracy                           0.00     287.0
   macro avg       0.00      0.00      0.00     287.0
weighted avg       0.00      0.00      0.00     287.0



C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defi

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00     287.0

    accuracy                           0.00     287.0
   macro avg       0.00      0.00      0.00     287.0
weighted avg       0.00      0.00      0.00     287.0



C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       286

    accuracy                           1.00       286
   macro avg       1.00      1.00      1.00       286
weighted avg       1.00      1.00      1.00       286



C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       286

    accuracy                           1.00       286
   macro avg       1.00      1.00      1.00       286
weighted avg       1.00      1.00      1.00       286

Modelprediction-Accuracy: 0.7585; Standard Deviation: 0.3982
Recall: 0.0000; Standard Deviation: 0.0000
F1-score: 0.0000; Standard Deviation: 0.0000
G-means: 0.6000; Standard Deviation: 0.4899


C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [21]:
#sample weight
def Catclass(clas, X, target):
    acc = []
    rec = []
    f1 = []
    gm = []
    mae = []
    for train, test in kf.split(X,target): 
        X_train = X.loc[train]
        y_train = target.loc[train]    
        
    
        
        #UNDERSAMPLING
        
        #TomekLinks
        #utl = TomekLinks() 
        #usa_X,usa_y = utl.fit_sample(X_train, y_train)
        
        
      
        # The proportion of categories before resampling
        #print(y_train.value_counts()/len(y_train))
        # The proportion of categories after resampling
        #print('After resampling:')
        #print(pd.Series(usa_y).value_counts()/len(usa_y))
       
        
        
        X_test = X.loc[test]
        y_test = target.loc[test]

        #accuracy of model prediction and other metrics
       
        #clas.fit(usa_X,usa_y) # undersampling 
        
        '''The “balanced” mode uses the values of y to automatically adjust
        weights inversely proportional to class frequencies in the input data: n_samples / (n_classes * np.bincount(y)).'''
        sample_weights = class_weight.compute_sample_weight('balanced', y_train)
        
        clas.fit(X_train, y_train, sample_weight = sample_weights)
        pred = clas.predict(X_test)
        print(metrics.classification_report(y_test, pred))
        acc.append(metrics.accuracy_score(y_test, pred))
        rec.append(metrics.recall_score(y_test, pred))
        f1.append(metrics.f1_score(y_test, pred)) 
        gm.append(geometric_mean_score(y_test, pred))
        #mae.append(mean_absolute_error(y_test, pred))
    print('Modelprediction-Accuracy: %.4f; Standard Deviation: %.4f' % (np.mean(acc), np.std(acc)))
    print('Recall: %.4f; Standard Deviation: %.4f' % (np.mean(rec), np.std(rec)))
    print('F1-score: %.4f; Standard Deviation: %.4f' % (np.mean(f1), np.std(f1)))
    print('G-means: %.4f; Standard Deviation: %.4f' % (np.mean(gm), np.std(gm))) 

    #print('MAE: %.4f; Standard Deviation: %.4f' % (np.mean(mae), np.std(mae)))
#     results = cross_val_score(clas, X, target, cv=kf, scoring='neg_mean_absolute_error')
#     print(results)
#     print("Accuracy: %.4f (%.4f)" % (results.mean(), results.std()))

In [22]:
Catclass(RandomForestClassifier(random_state = 67), ds[tr],ds.symptom)

C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defi

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00     287.0

    accuracy                           0.00     287.0
   macro avg       0.00      0.00      0.00     287.0
weighted avg       0.00      0.00      0.00     287.0



C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defi

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00     287.0

    accuracy                           0.00     287.0
   macro avg       0.00      0.00      0.00     287.0
weighted avg       0.00      0.00      0.00     287.0



C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       286

    accuracy                           1.00       286
   macro avg       1.00      1.00      1.00       286
weighted avg       1.00      1.00      1.00       286



C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       286

    accuracy                           1.00       286
   macro avg       1.00      1.00      1.00       286
weighted avg       1.00      1.00      1.00       286

Modelprediction-Accuracy: 0.7585; Standard Deviation: 0.3982
Recall: 0.0000; Standard Deviation: 0.0000
F1-score: 0.0000; Standard Deviation: 0.0000
G-means: 0.6000; Standard Deviation: 0.4899


C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Py\lib\site-packages\sklearn\metrics\_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
